In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

## Data Cleaning part 2 and EDA

In [2]:
df = pd.read_parquet("../data/usa_00001_clean.parquet")

df.columns

Index(['YEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'STATEFIP',
       'STRATA', 'GQ', 'PERNUM', 'PERWT', 'SEX', 'AGE', 'MARST', 'RACE',
       'RACED', 'BPL', 'BPLD', 'CITIZEN', 'YRIMMIG', 'LANGUAGE', 'LANGUAGED',
       'SPEAKENG', 'EDUC', 'EDUCD', 'EMPSTAT', 'EMPSTATD', 'LABFORCE',
       'CLASSWKR', 'CLASSWKRD', 'OCC', 'IND', 'WKSWORK2', 'UHRSWORK',
       'FTOTINC', 'INCWAGE'],
      dtype='object')

In [8]:
df_cleaned = df[(df["INCWAGE"] > 0) & df["EMPSTAT"].isin([1, 2])].copy() # Only people with positive income and either employeed or unemployeed
df_cleaned['YEARS_IN_US'] = df_cleaned['YEAR'] - df_cleaned['YRIMMIG']
df_cleaned = df_cleaned[["AGE", "SEX", "RACE", "MARST", "STATEFIP", # Demographics
                         "YEARS_IN_US", "EDUC", "OCC", "IND", # Personal info
                         "PERWT", # Personal Weight, may be helpful
                         "INCWAGE", # Treatment
                         "CITIZEN", # Outcome
                         ]]
df_cleaned = df_cleaned[df_cleaned["CITIZEN"].isin([2, 3])]
df_cleaned["CITIZEN"] = df_cleaned["CITIZEN"].map({2:1, 3:0}) # 1 if naturalized
df_cleaned.head()

,AGE,SEX,RACE,MARST,STATEFIP,YEARS_IN_US,EDUC,OCC,IND,PERWT,INCWAGE,CITIZEN
842,49,2,6,6,1,47,6,8800,3470,5.0,7000.0,1
1125,49,2,6,6,1,47,6,8800,3470,36.0,7000.0,1
1875,49,2,6,6,1,47,6,8800,3470,38.0,7000.0,1
2945,53,2,7,1,1,43,3,9600,6390,108.0,37400.0,1
2946,46,1,7,1,1,23,0,4230,7690,115.0,360.0,0


In [9]:
print(df_cleaned.shape)

(223823, 12)


In [10]:
print(len(df_cleaned.OCC.unique()))
print(len(df_cleaned.IND.unique()))

529
264


In [14]:
df_cleaned["INCWAGE_LOG"] = np.log(df_cleaned['INCWAGE'])
categorical_vars = ['SEX', 'RACE', 'MARST', 'EDUC', 'STATEFIP', 'OCC', 'IND']
min_count = 20

for col in categorical_vars:
    counts = df_cleaned[col].value_counts()
    rare = counts[counts < min_count].index
    df_cleaned[col] = df_cleaned[col].replace(rare, "OTHER")

treat = df_cleaned[df_cleaned["CITIZEN"] == 1]
control = df_cleaned[df_cleaned["CITIZEN"] == 0]

mask = np.ones(len(df_cleaned), dtype=bool)

for col in categorical_vars:
    treat_cats = set(treat[col].unique())
    control_cats = set(control[col].unique())
    common = treat_cats & control_cats
    mask &= df_cleaned[col].isin(common)

df_cleaned = df_cleaned[mask].copy()

for col in categorical_vars:
    df_cleaned[col] = df_cleaned[col].astype("category")

df_cleaned.head()

,AGE,SEX,RACE,MARST,STATEFIP,YEARS_IN_US,EDUC,OCC,IND,PERWT,INCWAGE,CITIZEN,INCWAGE_LOG
842,49,2,6,6,1,47,6,8800,3470,5.0,7000.0,1,8.853665
1125,49,2,6,6,1,47,6,8800,3470,36.0,7000.0,1,8.853665
1875,49,2,6,6,1,47,6,8800,3470,38.0,7000.0,1,8.853665
2945,53,2,7,1,1,43,3,9600,6390,108.0,37400.0,1,10.529426
2946,46,1,7,1,1,23,0,4230,7690,115.0,360.0,0,5.886104


In [16]:
df_cleaned.shape

(223798, 13)

In [15]:
df_cleaned.to_csv("../data/cleaned.csv", index = False)